# Importing the Libraries

In [ ]:
import collections.abc
import collections
import cv2
import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import pycocotools
import pylab
import re
import shutil
import tensorflow as tf
import time as time
import wget
import zipfile

from cv2 import dnn_superres
from dict2xml import dict2xml
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from PIL import Image
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from time import time

from Cluster import *
from ALAF import *
from Utils import *

In [ ]:
#PLEASE DEFINE WHERE DO YOU WANT TO CREATE THE FOLDER TO SAVE THE OUTPUTS PROVIDE BY OUR PROPOSAL
OUTPUT_DIR = ''
create_dir(OUTPUT_DIR)

# Download Super-Resolution Model

In [ ]:
MODEL_NAME = 'FSRCNN_x2'
MODEL_SR_DIR = OUTPUT_DIR+'MODEL_SR'
create_dir(MODEL_SR_DIR)
PATH_TO_MODEL_DIR = download_model_SR(MODEL_NAME, MODEL_SR_DIR+'/FSRCNN_x2.pb')

# Download Object Detection Model - EfficientDet D4

In [ ]:
MODEL_DATE = '20200711'
MODEL_NAME = 'efficientdet_d4_coco17_tpu-32'
PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)

# Download the labels

In [ ]:
LABEL_FILENAME = 'mscoco_complete_label_map.pbtxt'
PATH_TO_LABELS = download_labels(LABEL_FILENAME)

# Loading the Object Detection Model

In [ ]:
import time
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
print('Loading model...', end='')
start_time = time.time()
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# Loading Labels

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Start Our Proposal

In [ ]:
matplotlib.use('Agg')

In [ ]:
dict_COCO_names = {
    1:"person", 2:"bicycle", 3:"car", 4:"motorcycle", 5:"airplane", 6:"bus", 7:"train", 8:"truck", 9:"boat", 10:"traffic light", 11:"fire hydrant", 12:"street sign", 13:"stop sign", 14:"parking meter", 15:"bench", 16:"bird", 17:"cat", 18:"dog", 19:"horse", 20:"sheep", 21:"cow", 22:"elephant", 23:"bear", 24:"zebra", 25:"giraffe", 26:"hat", 27:"backpack", 28:"umbrella", 29:"shoe", 30:"eye glasses", 31:"handbag", 32:"tie", 33:"suitcase", 34:"frisbee", 35:"skis", 36:"snowboard", 37:"sports ball", 38:"kite", 39:"baseball bat", 40:"baseball glove", 41:"skateboard", 42:"surfboard", 43:"tennis racket", 44:"bottle", 45:"plate", 46:"wine glass", 47:"cup", 48:"fork", 49:"knife", 50:"spoon", 51:"bowl", 52:"banana", 53:"apple", 54:"sandwich", 55:"orange", 56:"broccoli", 57:"carrot", 58:"hot dog", 59:"pizza", 60:"donut", 61:"cake", 62:"chair", 63:"couch", 64:"potted plant", 65:"bed", 66:"mirror", 67:"dining table", 68:"window", 69:"desk", 70:"toilet", 71:"door", 72:"tv", 73:"laptop", 74:"mouse", 75:"remote", 76:"keyboard", 77:"cell phone", 78:"microwave", 79:"oven", 80:"toaster", 81:"sink", 82:"refrigerator", 83:"blender", 84:"book", 85:"clock", 86:"vase", 87:"scissors", 88:"teddy bear", 89:"hair drier", 90:"toothbrush", 91:"hair brush"
}

# Test Our Proposal Without Fine Tuning

In [ ]:
#Arguments: make_inference_SRSR(model_loaded,category_index,path_image_infer,name_image,path_save_image_with_detections,clases_to_detect,min_score_detect_element,path_model_sr)

In [ ]:
make_inference_SRSR(detect_fn,category_index,'/opt/share/Github/TEST/0.jpg','OUT.jpg','/opt/share/Github/TEST/PRUEBA',[3],0.35,'/opt/share/Github/TEST/MODEL_SR/')

In [ ]:
%%capture

#PLEASE INSERT THE PATH OF THE FOLDER THAT CONTAINS THE IMAGES TO TRAIN
imagenes_dir_train = ''

#PLEASE INSERT THE CLASSES (COCO) THAT YOU WANT TO DETECT FOR EXAMPLE, IF YOU WANT TO DETECT ONLY CARS [3]
CLASES = []

MODEL = 'OUTPUT'

unlabeled_dataset = os.listdir(imagenes_dir_train)

create_dir(OUTPUT_DIR+MODEL)
create_dir(OUTPUT_DIR+MODEL+'/FACTOR-OPT/AUX/')
create_dir(OUTPUT_DIR+MODEL+'/json/')
create_dir(OUTPUT_DIR+MODEL+'/json/train')
create_dir(OUTPUT_DIR+MODEL+'/json/validation')
create_dir(OUTPUT_DIR+MODEL+'/xml/')
create_dir(OUTPUT_DIR+MODEL+'/xml/train')
create_dir(OUTPUT_DIR+MODEL+'/xml/validation')
create_dir(OUTPUT_DIR+MODEL+'/AUX/')
create_dir(OUTPUT_DIR+MODEL+'/FT/')
create_dir(OUTPUT_DIR+MODEL+'/FT/MODEL')
create_dir(OUTPUT_DIR+MODEL+'/FT/EXPORT')

counter = 0
PATH_AUX = OUTPUT_DIR+MODEL+'/AUX/'

'''SPLITTING OF IMAGES FOR TRAINING AND VALIDATION'''
Number_Images_Split = int(len(unlabeled_dataset) * 15 / 100)
limite = int(len(unlabeled_dataset)-Number_Images_Split)
counter_split = 0
    
for frame in unlabeled_dataset:
   
  if int(counter_split) < limite:
    frame_path_copy = OUTPUT_DIR+MODEL+'/xml/train'+"/{}".format(frame)
    json_dir = OUTPUT_DIR+MODEL+'/json/train'+'/'
    xml_dir = OUTPUT_DIR+MODEL+'/xml/train'+'/'
  else:
    frame_path_copy = OUTPUT_DIR+MODEL+'/xml/validation'+"/{}".format(frame)
    json_dir = OUTPUT_DIR+MODEL+'/json/validation'+'/'
    xml_dir = OUTPUT_DIR+MODEL+'/xml/validation'+'/'
     
  counter_split = counter_split + 1
  
  folder_save_detections = OUTPUT_DIR+MODEL+'/FACTOR-OPT/'
  image_path_normal = imagenes_dir_train+str(frame)
  shutil.copyfile(image_path_normal, frame_path_copy)
  
  output = make_inference_SRSR(detect_fn,category_index,frame_path_copy,frame,folder_save_detections,CLASES,0.25, MODEL_SR_DIR+'/')
  counter = counter+1

  clases = output[2]
  width = output[3]
  height = output[4]

  id = frame.replace('.jpg','')

  x = '{"annotation":{"folder": "images","filename": "1.jpg","path": "/ALAF/images/1.jpg","source": {"database": "Unknown"},"size": {"width": "0","height": "0","depth": "3"},"segmented": "0","object": []}}'
  y = json.loads(x)
  y["annotation"]["folder"] = 'IMAGENES'
  y["annotation"]["filename"] = frame
  y["annotation"]["path"] = frame_path_copy
  y["annotation"]["size"]["width"] = width
  y["annotation"]["size"]["height"] = height

  boxes2 = []
    
  for id_class, box in enumerate(output[0]):
    ymin = int(box[0]*height)
    xmin = int(box[1]*width)
    ymax = int(box[2]*height)
    xmax = int(box[3]*width) 

    objeto1 = '{"name": "car","pose": "Unspecified","truncated": "0","difficult": "0","bndbox": {"xmin": "302","ymin": "215","xmax": "338","ymax": "237"}}'
    z = json.loads(objeto1)
    
    z["name"] = str(dict_COCO_names[clases[id_class]])
    z["bndbox"]["xmin"] = xmin 
    z["bndbox"]["xmax"] = xmax
    z["bndbox"]["ymin"] = ymin
    z["bndbox"]["ymax"] = ymax
    y['annotation']['object'].append(z)
    
  json_file_path  = json_dir+id+'.json'
  xml_file_path   = xml_dir+id+'.xml'
  
  with open(json_file_path, 'w', encoding='utf-8') as f:
    json.dump(y, f)

  if len(output[0]) > 0:
      with open(json_file_path) as json_file:
        data = json.load(json_file)
        xml = dict2xml(data)
        with open(xml_file_path, "w") as f:
          f.write(xml)
  else:
    os.remove(frame_path_copy)
    

In [ ]:
!python /opt/share/DATA/generate_tfrecord.py -x OUTPUT_DIR+MODELO+'/xml/train/' -l /opt/share/DATA/REENTRENOSM2/label_map.pbtxt -o OUTPUT_DIR+MODELO+'/xml/train/trainOK.record'

In [ ]:
!python /opt/share/DATA/generate_tfrecord.py -x OUTPUT_DIR+MODELO+'/xml/validation/' -l /opt/share/DATA/REENTRENOSM2/label_map3.pbtxt -o OUTPUT_DIR+MODELO+'/xml/validation/validationOK.record'

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d4_coco17_tpu-32.tar.gz
!tar xvzf efficientdet_d4_coco17_tpu-32.tar.gz

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python /home/ivangarcia/Desktop/PRL/DATA/model_main_tf2.py --model_dir=/home/ivangarcia/Desktop/PRL/MODELOS/PRUEBASVIDEO2/2/MODELORE/ --pipeline_config_path=/home/ivangarcia/Desktop/PRL/MODELOS/PRUEBASVIDEO2/2/ED4.config

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python /home/ivangarcia/Desktop/PRL/DATA/exporter_main_v2.py --input_type image_tensor --pipeline_config_path /home/ivangarcia/Desktop/PRL/MODELOS/PRUEBASVIDEO2/1/ED4.config --trained_checkpoint_dir /home/ivangarcia/Desktop/PRL/MODELOS/PRUEBASVIDEO2/1/EXP/ --output_directory /home/ivangarcia/Desktop/PRL/MODELOS/PRUEBASVIDEO2/1/SV/

# EVALUATION OF THE RE-TRAINED MODEL

In [ ]:
#PLEASE INSERT THE PATH OF THE FINE-TUNED MODEL
PATH_TO_MODEL_DIR = OUTPUT_DIR+MODEL+'/FT/EXPORT/'

In [ ]:
import time
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
print('Loading model...', end='')
start_time = time.time()
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))


# LOAD THE JSON WITH THE GT ANNOTATIONS

In [ ]:
import json
jsonString = ''

diccionario_ids = {}

with open(jsonString) as json_file:
    data = json.load(json_file)
    for p3 in data['images']:
        data_s_image = p3['file_name'].replace('images/','')
        data_s_id = p3['id']
        diccionario_ids[data_s_image] = data_s_id

In [ ]:
%%capture

#PLEASE INSERT THE PATH OF THE FOLDER THAT CONTAINS THE IMAGES TO TEST THE FINE-TUNED MODEL
imagenes_dir_train = ''

#PLEASE INSERT THE PATH TO GENERATE THE OUTPUTS
PATH_EVAL = ''

#PLEASE INSERT THE CLASSES (COCO) THAT YOU WANT TO DETECT
CLASES = []

#PLEASE INSERT THE MINIMUN SCORE TO DETECT ONE OBJECT
SCORE = 0.35

MODEL = 'OUTPUT-EVAL'

test_dataset = os.listdir(imagenes_dir_train)

create_dir(PATH_EVAL+MODEL)
create_dir(PATH_EVAL+MODEL+'/FACTOR-OPT/SALIDA/')
create_dir(PATH_EVAL+MODEL+'/AUX/')
create_dir(PATH_EVAL+MODEL+'/JSON/')
PATH_AUX = PATH_EVAL+MODEL+'/AUX/'

imagenes_dir3 = imagenes_dir_train
frames_test = os.listdir(imagenes_dir_train)
imagenes_dir2 = []
counter = 0

result = []
result2 = []
out = []
out2 = []


for frame in frames_test:
  
  id = frame.replace('.jpg','')
  image_path_save = imagenes_dir_train+"/"+str(frame)
  
  output = make_inference_SRSRRAW(detect_fn,category_index,image_path_save,frame,PATH_EVAL+MODEL+'/AUX/',CLASES,SCORE)

  counter = counter+1
    
  result = []
  result2 = []
  out2= []
    
  '''GET THE ID FOR EACH FRAME'''
  converted_num = diccionario_ids[frame]
    
  width = output[3]
  height = output[4]
  boxes2 = []


  '''TRANSLATE THE COORDINATES WITH COCO FORMAT'''
  for box in output[0]:
    ymin = int(box[0]*height)
    xmin = int(box[1]*width)
    ymax = int(box[2]*height)
    xmax = int(box[3]*width) 

    box_new = []
    box_new.append(xmin)
    box_new.append(ymin)
    box_new.append(xmax-xmin)
    box_new.append(ymax-ymin)
  

    boxes2.append(box_new)
  
  
  result.extend(
        [
            {
                "image_id": converted_num,
                "category_id": int(output[2][k]),
                "bbox": box,
                "score": output[1][k].astype(float),
            }
            for k, box in enumerate(boxes2)
        ]
  )

  result2.extend(
        [
            {
                "image_id": converted_num,
                "category_id": int(output[2][k]),
                "bbox": box,
                "score": output[1][k].astype(float),
            }
            for k, box in enumerate(boxes2)
        ]
  )


  for save_file in result2:    
        out2.append(save_file)
        
  json_file = PATH_EVAL+MODEL+'/JSON/'+str(converted_num)+'.json'
  with open(json_file, 'w', encoding='utf-8') as save_files:
    json.dump(out2, save_files, ensure_ascii=False)
        
  for sample in result:    
        out.append(sample)


with open(PATH_EVAL+MODEL+'/'+'test_data_modificado.json', 'w', encoding='utf-8') as file:
  json.dump(out, file, ensure_ascii=False)

In [ ]:
# pylab.rcParams['figure.figsize'] = (10.0, 8.0)

annType = ['segm','bbox','keypoints']
annType = annType[1]      #specify type here
prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
print ('Running demo for *%s* results.'%(annType))

'''LOAD THE JSON WITH THE GROUND TRUTH (GT)'''
annFile = ''
cocoGt=COCO(annFile)

'''LOAD THE JSON GENERATE IN THE PREVIOUS STEP'''
resFile = ''

cocoDt=cocoGt.loadRes(resFile)
dts = json.load(open(resFile,'r'))
imgIds = [imid['image_id'] for imid in dts]
imgIds = sorted(list(set(imgIds)))

cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.imgIds  = imgIds

'''PLEASE INSERT THE CLASS THAT YOU WANT TO EVALAUTE'''
cocoEval.params.catIds = [3] 
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()